In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import time

pref_code = [46] #都道府県コード
block_code = [1008] #地域コード
place_name = ["小田原"]

/Users/abeyuuya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# DBファイルを保存するためのファイルパス

import sqlite3

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect("weather.sqlite")
cur = con.cursor()

# テーブルが存在しない場合は作成
cur.execute('''CREATE TABLE IF NOT EXISTS weather
                (date TEXT, precipitation FLOAT, temp_avg FLOAT, temp_high FLOAT, temp_low FLOAT, humidity_avg FLOAT, humidity_min FLOAT, sunshine FLOAT)''')

# DBへの接続を閉じる
con.commit()
con.close()

In [10]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=46&block_no1008&year=2023&month=12&day=&view=p1"

# 取ったデータをfloat型に変える(データが取れなかった場合、気象庁は"/"を埋め込んでいるため0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0

In [11]:
if __name__ == "__main__":
  for place in place_name:
    #最終的にデータを集めるリスト
    list = [['年月日', '降水量', '気温_平均', '気温_最高', '気温_最低', '湿度_平均', '湿度_最小', '日照時間']]
    print(place)
    index = place_name.index(place)

    year = 2023
    print(year)

    month = 12
    # 2つの都市コードと年と月を当てはめる
    r = requests.get(url % (pref_code, block_code, year, month))
    r.encoding = r.apparent_encoding

    # スクレイピング
    soup = BeautifulSoup(r.text, 'html.parser')
    # findAllで条件に一致するものをすべて抜き出す。
    # 今回の条件はtrタグでclassがmtxになっているもの。
    rows = soup.findAll('tr', class_='mtx')
    # 表の最初の4行目はカラム情報なのでスライスする
    rows = rows[4:]

    # データベースに接続
    con = sqlite3.connect("weather.sqlite")
    cur = con.cursor()

    # 対象期間の1行を取得し、リストに挿入
    for row in rows:
      # trのなかのtdをすべて抜き出す
      data = row.findAll('td')

      # データを取り出す
      rowData = [] # 初期化
      time.sleep(1)
      rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
      rowData.append(str2float(data[3].string))
      rowData.append(str2float(data[6].string))
      rowData.append(str2float(data[7].string))
      rowData.append(str2float(data[8].string))
      rowData.append(str2float(data[9].string))
      rowData.append(str2float(data[10].string))
      rowData.append(str2float(data[16].string))

      # 次の行にデータを追加
      list.append(rowData)

      # データベースへの挿入
      cur.execute("INSERT INTO weather (date, precipitation, temp_avg, temp_high, temp_low, humidity_avg, humidity_min, sunshine)VALUES (?, ?, ?, ?, ?, ?, ?, ?)", rowData)

    con.commit()
    con.close()

return rowData

小田原
2023


TypeError: not all arguments converted during string formatting

In [ ]:
# DBに接続する
con = sqlite3.connect("weather.sqlite")
cur = con.cursor()

# SQLを用意
sql_select = "SELECT * FROM weather;"

# SQLを実行
cur.execute(sql_select)

for row in cur:
    print(row)

# DBへの接続を閉じる
con.close()